In [1]:
library(ArchR)
library(org.Ss.eg.db)
library(SuscrofaTxdb.11.108.fixed)
library(GenomicRanges)
library(gprofiler2)
library(GenomicFeatures)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

In [3]:
txdb_double <- makeTxDbFromGFF("/home/adufour/work/genome_and_annotation/plus4pigs.mapped_to_reference.2023_01_18.Sus_scrofa.Sscrofa11.1.108.sorted.gtf", format="gtf")

Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
Warning message in .get_cds_IDX(mcols0$type, mcols0$phase):
“The "phase" metadata column contains non-NA values for features of type
  stop_codon. This information was ignored.”
OK



In [5]:
genes <- GenomicFeatures::genes(txdb_double)

In [6]:
dataframe <- gconvert(
genes$gene_id,
organism = "sscrofa",
target = "ENSG",
mthreshold = 1,
filter_na = FALSE
)

In [7]:
genes$gene_id <- dataframe$target

In [8]:
genes$symbol <- dataframe$name

In [9]:
names(genes) <- NULL

In [10]:
genes <- genes[!genes$gene_id %in% NA]

In [11]:
filter <- seqlevels(genes)[!seqlevels(genes) %in% seqlevelsInUse(genes)]

In [12]:
seqlevels(genes) <- seqlevelsInUse(genes)

In [13]:
exons <- exons(txdb_double, columns="gene_id")

In [14]:
dataframe <- gconvert(
exons$gene_id,
organism = "sscrofa",
target = "ENSG",
mthreshold = 1,
filter_na = FALSE
)

In [15]:
exons$gene_id <- dataframe$target

In [16]:
exons$symbol <- dataframe$name

In [17]:
exons <- exons[!exons$gene_id %in% NA]

In [18]:
seqlevels(exons) <- seqlevelsInUse(exons)

In [19]:
transcripts_list <- transcripts(txdb_double, columns=c("tx_id", "tx_name"))

In [20]:
transcripts_list <- transcripts_list[seqnames(transcripts_list) %in% seqlevels(transcripts_list)[seqlevels(transcripts_list) %in% seqlevels(genes)]]

In [21]:
seqlevels(transcripts_list) <- seqlevelsInUse(transcripts_list)

In [22]:
tss <- resize(transcripts_list, width=1, fix='start')

In [23]:
geneAnnotation <- createGeneAnnotation(genes = genes, exons = exons, TSS = tss, OrgDb = org.Ss.eg.db)

In [25]:
saveRDS(geneAnnotation, file = "/home/adufour/work/rds_storage/omics/geneannotation.rds")